In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [3]:
test_input=pd.read_csv('C:\\Users\\ADMIN\\Desktop\\Credit_Risk_Validate_data.csv')

In [4]:
train_input=pd.read_csv('C:\\Users\\ADMIN\Desktop\\Credit_Risk_Train_data.csv')

In [5]:
print(train_input.columns)
print(test_input.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'outcome'],
      dtype='object')


In [6]:
# the last column has a different name in both,
# lets make the name same and merge them together
# so that we can fill the missing valve simultenusely
test_input.rename(columns={'outcome':'Loan_Status'},inplace=True)

In [7]:
data_all=pd.concat([train_input,test_input],axis=0)
data_all.shape

(981, 13)

In [8]:
print(data_all.tail())
data_all.reset_index(inplace=True,drop=True)
# reset index else merging will have issues

      Loan_ID Gender Married Dependents     Education Self_Employed  \
362  LP002971   Male     Yes         3+  Not Graduate           Yes   
363  LP002975   Male     Yes          0      Graduate            No   
364  LP002980   Male      No          0      Graduate            No   
365  LP002986   Male     Yes          0      Graduate            No   
366  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
362             4009             1777.0       113.0             360.0   
363             4158              709.0       115.0             360.0   
364             3250             1993.0       126.0             360.0   
365             5000             2393.0       158.0             360.0   
366             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
362             1.0         Urban           Y  
363             1.0         Urban     

In [9]:
print(data_all.tail())

      Loan_ID Gender Married Dependents     Education Self_Employed  \
976  LP002971   Male     Yes         3+  Not Graduate           Yes   
977  LP002975   Male     Yes          0      Graduate            No   
978  LP002980   Male      No          0      Graduate            No   
979  LP002986   Male     Yes          0      Graduate            No   
980  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
976             4009             1777.0       113.0             360.0   
977             4158              709.0       115.0             360.0   
978             3250             1993.0       126.0             360.0   
979             5000             2393.0       158.0             360.0   
980             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
976             1.0         Urban           Y  
977             1.0         Urban     

In [10]:
Counter(data_all['Gender'])

Counter({'Male': 775, 'Female': 182, nan: 24})

In [11]:
# lets fill them as male
print(data_all[data_all['Gender'].isnull()].index.tolist())
# these rows are null for gender
# lets fill them with model of gender i.e. male
gender_null=data_all[data_all['Gender'].isnull()].index.tolist()

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [12]:
data_all['Gender'].iloc[gender_null]='Male'

In [13]:
# check if failed
print(sum(data_all['Gender'].isnull()))# ok done
Counter(data_all['Gender'])


0


Counter({'Male': 799, 'Female': 182})

In [14]:
# lets fill married noe
print(Counter(data_all['Married']))

Counter({'Yes': 631, 'No': 347, nan: 3})


In [15]:
married_null=data_all[data_all['Married'].isnull()].index.tolist()
married_null

[104, 228, 435]

In [16]:
data_all['Married'].iloc[married_null]='Yes'

In [17]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [18]:
Counter(data_all['Dependents'])

Counter({'0': 545, '1': 160, '2': 160, '3+': 91, nan: 25})

In [19]:
# lets see the dependents wrt marrige
pd.crosstab(data_all['Married'],data_all['Dependents'].isnull())

Dependents,False,True
Married,,
No,338,9
Yes,618,16


In [20]:
pd.crosstab(data_all['Dependents'],data_all['Married'])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [21]:
# for bachelors lets fill the missing dependetns as 0
# lets find the index of all the rows with dependents missing and married no.
bachelor_nulldependent=data_all[(data_all['Married']== 'No') & (data_all['Dependents'].isnull())].index.tolist()
print(bachelor_nulldependent)

[293, 332, 355, 597, 684, 752, 879, 916, 926]


In [22]:
data_all['Dependents'].iloc[bachelor_nulldependent]='0'   # 0 string me hona jaroori hai

In [23]:
Counter(data_all['Dependents'])

Counter({'0': 554, '1': 160, '2': 160, '3+': 91, nan: 16})

In [24]:
# for the remaininh 16 missing dependents
# lets seee hoe many depends male and female have
pd.crosstab(data_all['Gender'],data_all['Dependents'])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [25]:
# so female have less dependence
# lets see the gender of 16 missing dependents
data_all['Gender'].iloc[data_all['Dependents'].isnull().index.tolist()]

0        Male
1        Male
2        Male
3        Male
4        Male
5        Male
6        Male
7        Male
8        Male
9        Male
10       Male
11       Male
12       Male
13       Male
14       Male
15       Male
16       Male
17     Female
18       Male
19       Male
20       Male
21       Male
22       Male
23       Male
24       Male
25       Male
26       Male
27       Male
28       Male
29     Female
        ...  
951      Male
952      Male
953    Female
954      Male
955      Male
956      Male
957      Male
958      Male
959      Male
960      Male
961      Male
962      Male
963      Male
964      Male
965      Male
966      Male
967      Male
968      Male
969      Male
970      Male
971      Male
972      Male
973      Male
974    Female
975      Male
976      Male
977      Male
978      Male
979      Male
980      Male
Name: Gender, Length: 981, dtype: object

In [52]:
pd.crosstab((data_all['Gender']=='Male') & (data_all['Married']=='Yes'),data_all['Dependents'])

Dependents,0,1,2,3+
row_0,,,,
False,318,48,23,15
True,236,112,137,76


In [54]:
# lets fill the dependent with 1
data_all['Dependents'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]='1'

In [55]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [56]:
Counter(data_all['Self_Employed'])

Counter({'No': 862, 'Yes': 119})

In [28]:
Self_emp_null=data_all[data_all['Self_Employed'].isnull()].index.tolist()

In [30]:
#fill missing selfemployed with NO
data_all['Self_Employed'].iloc[Self_emp_null]='No'


In [31]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           16
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [33]:
# to check if any row with both loanamount and loan_amount_term as NAN
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'].isnull())

Loan_Amount_Term,False,True
LoanAmount,,
False,934,20
True,27,0


In [35]:
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [37]:
data_all.groupby(data_all['Loan_Amount_Term'])['LoanAmount'].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [38]:
# lets fill the  missing value in the loanamount
#with the mean of the respective loan_term
#we see thst 180& 240 has almost same loan amount 128-131
# and 360 has high i.e 144
#and all reameining by 130
data_all['LoanAmount'][(data_all['LoanAmount'].isnull()) & 
                      (data_all['Loan_Amount_Term']==360)]=144
data_all['LoanAmount'][(data_all['LoanAmount'].isnull()) & 
                      (data_all['Loan_Amount_Term']==480)]=137


In [39]:
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())]=130

In [40]:
# lets fill the loan amount term
(data_all['Loan_Amount_Term']).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
36.0       3
60.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [41]:
# lets fill the loan tenure by the mode i.e. 360
data_all['Loan_Amount_Term'][data_all['Loan_Amount_Term'].isnull()]=360

In [42]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           16
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [43]:
data_all['Credit_History'].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [44]:
pd.crosstab(data_all['Gender'],data_all['Credit_History'])
# gender makes no diiference

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [45]:
pd.crosstab(data_all['Self_Employed'],data_all['Credit_History'])
# self employes make no diffference

Credit_History,0.0,1.0
Self_Employed,,
No,134,658
Yes,14,96


In [46]:
pd.crosstab(data_all['Education'],data_all['Credit_History'])
# education make no difference

Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [47]:
pd.crosstab(data_all['Married'],data_all['Credit_History'])
# married makes no difference

Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [48]:
data_all['Credit_History'][data_all['Credit_History'].isnull()]=1

In [57]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [58]:
import pickle
    
fh=open('data_all.pkl','bw')
pickle.dump(data_all,fh)
fh.close()